In [76]:
import os
import importlib
import copy

In [77]:
import computer_science
importlib.reload(computer_science)
import req_builder
from req_builder import Req

from computer_science import TRACKS


[<req_builder.Req object at 0x0000022EA18C15C0>, <req_builder.Req object at 0x0000022EA18C1908>, <req_builder.Req object at 0x0000022EA18C14E0>, <req_builder.Req object at 0x0000022EA18C1D68>, <req_builder.Req object at 0x0000022EA18C1C18>, <req_builder.Req object at 0x0000022EA18C19E8>]
['CS221', <req_builder.Req object at 0x0000022EA18C1F28>, <req_builder.Req object at 0x0000022EA18C1400>, <req_builder.Req object at 0x0000022EA18C1160>]


### Put a sample course list of a student to test against majors like the one below 

In [11]:
samp_list = ['CHEM31A', 'CHEM31B', 'PHYSICS22', 'PHYSICS23', 'PHYSICS24', 'CHEM174', 'BIO86', 'CHEM121', 'CHEM131', 'CHEM151', 'CHEM171', 'CHEM181', 'MATH19', 'MATH20', 'MATH21', 'MATH51', 
             'PHYSICS42', 'PHYSICS43', 'PHYSICS44', 
    'CHEM123', 'CHEM124', 'CHEM126', 'CHEM185', 'CHEM173', 'CHEM183', 'CHEM184', 'CHEM176']

# Below is the current interation of the Major Progress Tool 
# _______________________________________________

### Assuming that they have all the requirements and if they don't populating a list with the requirements they don't have 

In [45]:
def major_progress_tracker(major, course_list, course_units):
    requirements_left = []
    reqs = TRACKS[major]
    for req in TRACKS[major]:
        if isinstance(req, Req):
            req_num = req.num[0]
            if req.num[1]:
                req_num = int(req_num/course_units)
            requirements_met = []
            requisite_list = []
            for requisite in req.reqs:
                if isinstance(requisite, Req):
                    inner_num = requisite.num[0]
                    if requisite.num[1]:
                        inner_num = int(inner_num/course_units)
                    inners_met = []
                    for inner_req in requisite.reqs:
                        if inner_req in course_list:
                            inners_met.append(inner_req)
                            inner_num -= 1
                        if inner_num == 0:
                            req_num -= 1
                    if inner_num > 0:
                        requires_left = set(requisite.reqs).difference(set(inners_met))
                        requisite_list.append((requires_left, inner_num))
                elif requisite in course_list:

                    requirements_met.append(requisite)

                    req_num -= 1
                else:
                    requisite_list.append(requisite)
                if req_num <= 0:
                    break
            if req_num > 0:
                if any(isinstance(x, Req) for x in req.reqs):
                    requirements_left.append((requisite_list, req_num))
                else:
                    reqs_left = set(req.reqs).difference(set(requirements_met))
                    requirements_left.append((reqs_left, req_num))
        elif req not in course_list:
            requirements_left.append(req)
    return requirements_left, str(round(len(requirements_left)/len(TRACKS[major]), 2) * 100) + '% more to go for the major: ' + str(major)

In [39]:
samp_req = Req(['CHEM31A', 'CHEM31B', 'PHYSICS22', 'PHYSICS23', 'Ki', Req(['KILLER230', 'PIRATES190', Req(['TEST IT FOR SURE'], (1, False))], (1, False))], (5, False))

In [64]:
def inner_req(req, courses, course_units):
    need = []
    req_num = req.num[0]
    if req.num[1]:
                req_num = int(req_num/course_units)
#     print(req.reqs)
    for requisite in req.reqs:
        if isinstance(requisite, Req):
            new_set = inner_req(requisite, courses, course_units)
            if len(new_set) > 0:
                need.append(new_set)
        elif requisite not in courses:
            need.append(requisite)
        else:
            req_num -= 1
    if req_num <= 0:
        return set()
    else:
        return (need, req_num) 

In [65]:
def major_progress_tracker_rev(major, course_list, course_units):
    requirements_left = []
    reqs = TRACKS[major]
    for req in TRACKS[major]:
        if isinstance(req, Req):
            requirements = inner_req(req, course_list,course_units)
            if len(requirements) > 0:
                requirements_left.append(requirements)
        elif req not in course_list:
            requirements_left.append(req)
    return requirements_left, str(round(len(requirements_left)/len(TRACKS[major]), 2) * 100) + '% more to go for the major: ' + str(major)

In [40]:
inner_req(samp_req, samp_list)

(['Ki', (['KILLER230', 'PIRATES190', (['TEST IT FOR SURE'], 1)], 1)], 1)

In [55]:
TRACKS['cs'][-1].reqs.reqs

AttributeError: 'list' object has no attribute 'reqs'

In [78]:
major_progress_tracker_rev('cs', samp_list, 5)

(['CS103',
  'CS109',
  (['MATH52',
    'MATH53',
    'MATH104',
    'MATH107',
    'MATH108',
    'MATH109',
    'MATH110',
    'MATH113',
    'CS157',
    'CS205L',
    'PHL151',
    'CME100',
    'CME102',
    'CME104',
    'ENGR108'],
   1),
  ([(['PHYSICS41', 'PHYSICS21', 'PHYSICS41E'], 1)], 2),
  (['AA252',
    'ARCHLGY151',
    'BIOE131',
    'CEE102A',
    (['CEE17X', 'CEE177S'], 2),
    'CLASSICS168',
    'COMM120W',
    'COMM166',
    'CS152',
    'CS181',
    'CS181W',
    'CS182',
    'CS182W',
    'CS278',
    'EARTHSYS125',
    'ENGR117',
    'ENGR145',
    'ENGR148',
    'HUMBIO174',
    'MS&E193',
    'ME167',
    'POLISCI114S',
    'PUBLPOL134',
    'STS1'],
   1),
  (['CS106B', 'CS106X'], 1),
  'ENGR40M',
  (['ENGR10',
    'ENGR14',
    'ENGR15',
    'ENGR20',
    'ENGR21',
    'ENGR40A',
    'ENGR40B',
    'ENGR40M',
    'ENGR42',
    'ENGR50',
    'ENGR50E',
    'ENGR50M',
    'ENGR60',
    'ENGR62',
    'ENGR62X',
    'ENGR76',
    'CS106A',
    'CS106B',
    'CS10

### This is how to call the function to track major progression

In [46]:
# first param. is the major that is in TRACKS built in the computer_science.py file
# Second param is the sample_course_list to test against
# third param is the number of units a class is worth (MUST put a value here even if major doesn't need it)

# for the nested reqs, make sure to just return the course left to take
# take the requirements --> substract from the requirement set after they have accomplished it
major_progress_tracker('history', samp_list, 5)


(['HISTORY13S',
  'HISTORY18S',
  'HISTORY23S',
  'HISTORY32S',
  'HISTORY47S',
  'HISTORY64S',
  'HISTORY82S',
  'HISTORY89S',
  'HISTORY98S',
  ({'HISTORY13S',
    'HISTORY18S',
    'HISTORY200A',
    'HISTORY200B',
    'HISTORY200C',
    'HISTORY200J',
    'HISTORY200M',
    'HISTORY200R',
    'HISTORY200Y',
    'HISTORY201A',
    'HISTORY201B',
    'HISTORY202G',
    'HISTORY203',
    'HISTORY203C',
    'HISTORY203F',
    'HISTORY204A',
    'HISTORY204D',
    'HISTORY204G',
    'HISTORY205L',
    'HISTORY210D',
    'HISTORY210J',
    'HISTORY211',
    'HISTORY216D',
    'HISTORY222',
    'HISTORY224C',
    'HISTORY227B',
    'HISTORY227K',
    'HISTORY228',
    'HISTORY232B',
    'HISTORY232G',
    'HISTORY233',
    'HISTORY233F',
    'HISTORY234P',
    'HISTORY235J',
    'HISTORY235L',
    'HISTORY23S',
    'HISTORY240',
    'HISTORY241C',
    'HISTORY242J',
    'HISTORY243C',
    'HISTORY243G',
    'HISTORY244F',
    'HISTORY248',
    'HISTORY251C',
    'HISTORY251J',
    'HISTOR

### Calculate the # of different ways to complete a major

In [6]:
TRACKS.keys()

dict_keys(['chem', 'biochem', 'bio', 'cheme', 'math', 'econ', 'history'])

In [7]:
def major_complexity(major, units): 
    count = 1
    for req in TRACKS[major]:
        if isinstance(req, Req):
            req_min = req.num[0]
            if req.num[1]:
                req_min /= units
            count *= len(req.reqs) // req_min
    return count

In [8]:
major_complexity('history', 3)

266